In [3]:
import sys
!{sys.executable} -m pip install nba_api

                                              0.0/257.3 kB ? eta -:--:--
     -------------------------------        215.0/257.3 kB 6.6 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     -----------------------------------  256.0/257.3 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 257.3/257.3 kB 608.4 kB/s eta 0:00:00
                                              0.0/62.6 kB ? eta -:--:--
     ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
                                              0.0/96.6 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from nba_api.stats.static import teams,players
from nba_api.stats.endpoints import playergamelog, playerindex
import requests
import pandas as pd
import time
import os

In [20]:
# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e,attempts)
                time.sleep(30)
                attempts += 1

    return retry_wrapper


@retry
def getAggPlayerStats(pid,fname,lname,position,df,minGames=50):
        
    #get player game logs    
    data = playergamelog.PlayerGameLog(player_id=pid,season='2022-23')
    data = data.player_game_log.get_data_frame()

    #remove less than n games
    if len(data) < minGames: return pd.DataFrame()
    
    # build row for player stats
    data = data[['MIN','FG_PCT','FT_PCT','FG3M','REB','AST','STL','BLK','PTS']]
    mean_df  = data.mean().to_frame().T
    mean_df.columns = [col + '_MEAN' for col in mean_df.columns]
    std_df = data.std().to_frame().T
    std_df.columns = [col + '_STD' for col in std_df.columns]

    # Join all into one table
    data = pd.concat([mean_df,std_df],axis=1)
    data['player_first_name'] = fname
    data['player_last_name'] = lname
    data['player_id'] = pid
    data['position'] = position

    #concat table to bigger df
    final_df = pd.concat([df,data])
    return final_df
        


         




In [21]:
df = pd.DataFrame()
players = playerindex.PlayerIndex().player_index.get_data_frame()
if os.path.exists('fantasy_stats.csv'):
    df = pd.read_csv('fantasy_stats.csv').drop('Unnamed: 0',axis=1)
    players = players[~players['PERSON_ID'].isin(df['player_id'])]

for _,player in players.iterrows():
    data = getAggPlayerStats(player['PERSON_ID'],player['PLAYER_FIRST_NAME'],player['PLAYER_LAST_NAME'],player['POSITION'],df)
    if len(data) > 0:
        df = data

df.to_csv('fantasy_stats.csv')

HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


In [23]:
df.to_csv('fantasy_stats.csv')
